<a name="TOP" />

# Networking / tcpdump tutorial

This tutorial will look at several networking tools available in the default Debian 'Jessie' repositories, especially tcpdump

**NOTE**: Although we cannot capture packets in this tutorial, we can use tcpdump to analyse sample capture files.  You can also upload your own pcap files for analysis, so this is really a live tutorial where you can fully experiment with tcpdump packet analysis/display/filtering capabilities

**NOTE: New to Jupyter?** - for information on how to use this notebook please refer to the <a href="INDEX_Tutorials.ipynb"> INDEX page </a>

### tcpdump man page

See <a href="#tcpdump_manpage"> tcpdump man page </a> reproduced below

### Resources

This tutorial was inspired by the tcpdump man pages and many online  <a href="#tcpdump_resources">resources</a> cited below



# What software versions are we using?

In the interest of documenting our experiment, let's first document what OS, what shell and other tool versions we are using

### OS version

In [28]:
echo -e "Running kernel version:\n\t" $(uname -a)
RELEASE_INFO=$(cat /etc/*release 2>/dev/null)
[ ! -z "$RELEASE_INFO" ] &&
    echo -e "Running release:"; sed 's/^/    /' /etc/*release

Running kernel version:
	 Linux notebook-server 3.16.0-0.bpo.4-amd64 #1 SMP Debian 3.16.7-ckt11-1+deb8u2~bpo70+1 (2015-07-22) x86_64 GNU/Linux
Running release:
    PRETTY_NAME="Debian GNU/Linux 8 (jessie)"
    NAME="Debian GNU/Linux"
    VERSION_ID="8"
    VERSION="8 (jessie)"
    ID=debian
    HOME_URL="http://www.debian.org/"
    SUPPORT_URL="http://www.debian.org/support/"
    BUG_REPORT_URL="https://bugs.debian.org/"


### Bash shell version

In [29]:
bash --version

GNU bash, version 4.3.30(1)-release (x86_64-pc-linux-gnu)
Copyright (C) 2013 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>

This is free software; you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


### tcpdump version

In [31]:
tcpdump --version

tcpdump version 4.6.2
libpcap version 1.6.2
OpenSSL 1.0.1k 8 Jan 2015


## Capturing with tcpdump

As we don't have permission to listen on network interfaces we will work with existing pcap capture files, already created with tcpdump.

Nevertheless, we first look at how to capture your own pcap files with tcpdump

On a linux system, such as Ubuntu, run
```/sbin/ip a```
command to see what network interfaces you have.

```
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 40:8d:5c:a0:a2:c8 brd ff:ff:ff:ff:ff:ff
    inet 192.168.0.34/24 brd 192.168.0.255 scope global dynamic eth0
       valid_lft 34763sec preferred_lft 34763sec
    inet6 fe80::30a7:9710:ffa6:eb21/64 scope link
       valid_lft forever preferred_lft forever
3: wlp2s0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc mq state DOWN group default qlen 1000
    link/ether b4:6d:83:d2:a2:f3 brd ff:ff:ff:ff:ff:ff
```

We see that my system has 2 networking interfaces
- a wired connection 'eth0'
- a wireless connection 'wlp2so'.

So to sniff for traffic on my wired connection I would enter the command

```tcpdump -i eth0```

**Note:** you will need to run this command as root, or under sudo

### capturing to a file

``` tcpdump -i eth0 -w sample.pcap```

Here is an example of capturing a pcap file on my system.
We can terminate the capture by interrupting by typing ctrl-C, or we can specify the maximum number of packets by adding the `*-c <count>*' parameters


### capturing until interruption:
using ``` tcpdump -i eth0 -w sample100.pcap```

<pre>
tcpdump: listening on eth0, link-type EN10MB (Ethernet), capture size 262144 bytes
10 packets captured
12 packets received by filter
0 packets dropped by kernel
mjb@NUC:~$ ll sample.pcap
-rw-r--r-- 1 root root 1126 Jul  9 08:12 sample.pcap
</pre>

### capturing 100 packets

using ``` tcpdump -v -i eth0 -c 100 -w sample100.pcap```

**Note:** we add -v option so that we see the number of packets captured whilst waiting for completion

<pre>
tcpdump: listening on eth0, link-type EN10MB (Ethernet), capture size 262144 bytes
100 packets captured
100 packets received by filter
0 packets dropped by kernel
mjb@NUC:~$ ll sample100.pcap
-rw-r--r-- 1 root root 10172 Jul  9 08:23 sample100.pcap
</pre>

### capturing on all interfaces

If you have multiple network interfaces available and you want to capture on all interfaces you can do this by specifying the ```-i any``` parameter, e.g.
using ``` tcpdump -v -i any```


## Analyzing packets

We could add many parameters to the above capture commands, but we will investigate their effects whilst reading pre-captured pcap files, here are the sample files we have available.

In [24]:
ls -al *.pcap

-rw-r--r-- 1 main main  3684 Jul  9 05:53 dhcpProblem.capture.pcap
-rw-r--r-- 1 main main  1126 Jul  9 06:38 sample.pcap
-rw-r--r-- 1 main main 10172 Jul  9 06:38 sample100.pcap


**NOTE:** As we are using pcap files from another system, we do not want tcpdump to perform any host name lookup - as it will delay processing considerably. To avoid hostname lookup we will add the -n parameter each time

First let's look at our initial sample capture file

### Reading captured packets from a file

We can read from our capture file using the ```-r``` parameter

In [32]:
tcpdump -n -r sample.pcap

reading from file sample.pcap, link-type EN10MB (Ethernet)
06:12:13.847128 IP 192.168.0.34.22 > 192.168.0.58.56441: Flags [P.], seq 2062217639:2062217763, ack 252119112, win 314, length 124
06:12:13.848798 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 124, win 53248, length 0
06:12:14.257627 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:14.965764 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:15.716589 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:16.466113 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:17.244502 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:17.855085 ARP, Request who-has 192.168.0.254 tell 192.168.0.20, length 46
06:12:17.967843 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:18.

### Showing hex and ascii dump of packets

Below we use the ```-A``` option to dump packets in ASCII, we limit to just thwe first 2 packets using ```-c 2```

In [35]:
tcpdump -n -A -c 2 -r sample.pcap

reading from file sample.pcap, link-type EN10MB (Ethernet)
06:12:13.847128 IP 192.168.0.34.22 > 192.168.0.58.56441: Flags [P.], seq 2062217639:2062217763, ack 252119112, win 314, length 124
E.....@.@......"...:...yz......HP..:....~..].........\N.xJ.~......kP...w.C..C-.W../.%..!..g..Y_/k.Mq7)n.'....X.k[....!....[.....`L..OAKG.. u....z..9._W.q..z....(...
06:12:13.848798 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 124, win 53248, length 0
E..(.@@...Y....:...".y.....Hz..#P....9........


Below we use the ```-XX``` option to dump packets in hexadecimal and in ASCII, we limit to just thwe first 2 packets using ```-c 2```

In [36]:
tcpdump -n -XX -c 2 -r sample.pcap

reading from file sample.pcap, link-type EN10MB (Ethernet)
06:12:13.847128 IP 192.168.0.34.22 > 192.168.0.58.56441: Flags [P.], seq 2062217639:2062217763, ack 252119112, win 314, length 124
	0x0000:  104a 7d2b bd67 408d 5ca0 a2c8 0800 4510  .J}+.g@.\.....E.
	0x0010:  00a4 eb81 4000 4006 cd15 c0a8 0022 c0a8  ....@.@......"..
	0x0020:  003a 0016 dc79 7aea f1a7 0f07 0848 5018  .:...yz......HP.
	0x0030:  013a 10d6 0000 7ee4 fe5d d98c 1cfa 1111  .:....~..]......
	0x0040:  e5c8 f05c 4eab 784a e67e 9c92 17fb a02e  ...\N.xJ.~......
	0x0050:  6b50 d2b6 0e77 dc43 f618 432d 9d57 849b  kP...w.C..C-.W..
	0x0060:  2ffa 2506 1721 0c97 67ac ea59 5f2f 6bb6  /.%..!..g..Y_/k.
	0x0070:  4d71 3729 6ecb 2787 de18 8258 b86b 5b99  Mq7)n.'....X.k[.
	0x0080:  acf3 aa21 fdc0 d5b3 5bbc 0f84 c3e6 604c  ...!....[.....`L
	0x0090:  b892 4f41 4b47 1e19 2075 e680 d0d2 7ae9  ..OAKG...u....z.
	0x00a0:  f339 825f 57b5 71f6 e37a 1a1d ee83 2804  .9._W.q..z....(.
	0x00b0:  e3a3                                     ..
06:12:13

## Filtering packets

### Saving a subset of packets to a file, by writing out to a file

Writing out just the first 2 packets:

In [105]:
tcpdump -n -c 2 -r sample.pcap -w saved2.pcap

reading from file sample.pcap, link-type EN10MB (Ethernet)


In [106]:
tcpdump -n -r saved2.pcap

reading from file saved2.pcap, link-type EN10MB (Ethernet)
06:12:13.847128 IP 192.168.0.34.22 > 192.168.0.58.56441: Flags [P.], seq 2062217639:2062217763, ack 252119112, win 314, length 124
06:12:13.848798 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 124, win 53248, length 0


###### Writing out only packets longer than 70 bytes

In [107]:
tcpdump -n -r sample.pcap greater 70 -w saved_70bytePackets.pcap

reading from file sample.pcap, link-type EN10MB (Ethernet)


In [108]:
tcpdump -n -r saved_70bytePackets.pcap

reading from file saved_70bytePackets.pcap, link-type EN10MB (Ethernet)
06:12:13.847128 IP 192.168.0.34.22 > 192.168.0.58.56441: Flags [P.], seq 2062217639:2062217763, ack 252119112, win 314, length 124
06:12:14.257627 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:14.965764 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:15.716589 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:16.466113 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:17.244502 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:17.967843 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST
06:12:18.717009 IP 192.168.0.58.137 > 192.168.0.255.137: NBT UDP PACKET(137): QUERY; REQUEST; BROADCAST


###### Writing out only packets shorter than 70 bytes

In [109]:
tcpdump -n -r sample.pcap less 70 -w saved_smallerThan70bytePackets.pcap

reading from file sample.pcap, link-type EN10MB (Ethernet)


In [110]:
tcpdump -n -r saved_smallerThan70bytePackets.pcap

reading from file saved_smallerThan70bytePackets.pcap, link-type EN10MB (Ethernet)
06:12:13.848798 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 2062217763, win 53248, length 0
06:12:17.855085 ARP, Request who-has 192.168.0.254 tell 192.168.0.20, length 46


###### Writing out just arp packets

In [78]:
tcpdump -n -r sample.pcap -w saved_ARP.pcap arp

reading from file sample.pcap, link-type EN10MB (Ethernet)


In [79]:
tcpdump -n -r saved_ARP.pcap

reading from file saved_ARP.pcap, link-type EN10MB (Ethernet)
06:12:17.855085 ARP, Request who-has 192.168.0.254 tell 192.168.0.20, length 46


In [80]:
ls -altr *.pcap

-rw-r--r-- 1 main main  3684 Jul  9 05:53 dhcpProblem.capture.pcap
-rw-r--r-- 1 main main  1126 Jul  9 06:38 sample.pcap
-rw-r--r-- 1 main main 10172 Jul  9 06:38 sample100.pcap
-rw-r--r-- 1 main main  1126 Jul  9 08:01 g_1024.pcap
-rw-r--r-- 1 main main  1126 Jul  9 08:01 g_1010.pcap
-rw-r--r-- 1 main main   294 Jul  9 08:01 saved2.pcap
-rw-r--r-- 1 main main  1126 Jul  9 08:01 g_10.pcap
-rw-r--r-- 1 main main  1126 Jul  9 08:01 g_0.pcap
-rw-r--r-- 1 main main   974 Jul  9 08:07 saved_70bytePackets.pcap
-rw-r--r-- 1 main main   100 Jul  9 08:08 saved_ARP.pcap


In [59]:
tcpdump -n  -r sample.pcap arp

reading from file sample.pcap, link-type EN10MB (Ethernet)
06:12:17.855085 ARP, Request who-has 192.168.0.254 tell 192.168.0.20, length 46


### Filtering packets by port number

In this example we filter on port 22 (ssh), by specifying the

<pre>port 22</pre>

filter, we select the first 2 packets.

In [118]:
tcpdump -n -r sample100.pcap -c 2 port 22

reading from file sample100.pcap, link-type EN10MB (Ethernet)
06:22:37.878076 IP 192.168.0.34.22 > 192.168.0.58.56441: Flags [P.], seq 2062226147:2062226271, ack 252123104, win 314, length 124
06:22:37.879741 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 124, win 52910, length 0


Note that in the above we see a source and a destination port 22.

If we want to see only packets with destination port 22, we specify the <pre>dst port 22</pre>
 filter below.  Note how we still get 2 packets, 

In [119]:
tcpdump -n -r sample100.pcap -c 2 dst port 22

reading from file sample100.pcap, link-type EN10MB (Ethernet)
06:22:37.879741 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 2062226271, win 52910, length 0
06:22:39.083785 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 45, win 52899, length 0


### Filtering packets by ip address and port number

In this example we filter on port 22 (ssh), and we will specify the source ip address we are interested in using the filter

<pre>src 192.168.0.34 and port 22</pre>

In [123]:
tcpdump -n -r sample100.pcap -c 2 src 192.168.0.34 and port 22

reading from file sample100.pcap, link-type EN10MB (Ethernet)
06:22:37.878076 IP 192.168.0.34.22 > 192.168.0.58.56441: Flags [P.], seq 2062226147:2062226271, ack 252123104, win 314, length 124
06:22:38.879339 IP 192.168.0.34.22 > 192.168.0.58.56441: Flags [P.], seq 124:168, ack 1, win 314, length 44


In this example we filter on port 22 (ssh), and we will specify the destination ip address we are interested in using the filter

<pre>dst 192.168.0.34 and port 22</pre>

In [126]:
tcpdump -n -r sample100.pcap -c 2 dst 192.168.0.34 and port 22

reading from file sample100.pcap, link-type EN10MB (Ethernet)
06:22:37.879741 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 2062226271, win 52910, length 0
06:22:39.083785 IP 192.168.0.58.56441 > 192.168.0.34.22: Flags [.], ack 45, win 52899, length 0


We can also combine filters to filter on multiple ports, now let's see how we can use this for a practical example

Let's look for dhcp exchanges (on port 67 or 68)

In [140]:
tcpdump -n -r dhcpProblem.capture.pcap port 67 or port 68

reading from file dhcpProblem.capture.pcap, link-type EN10MB (Ethernet)
18:11:26.463826 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from fa:16:3e:01:85:d1, length 280
18:11:26.464311 IP 10.0.0.1.67 > 10.0.0.4.68: BOOTP/DHCP, Reply, length 316
18:11:29.485856 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from fa:16:3e:01:85:d1, length 280
18:11:29.486185 IP 10.0.0.1.67 > 10.0.0.4.68: BOOTP/DHCP, Reply, length 316
18:11:32.506631 IP 0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from fa:16:3e:01:85:d1, length 280
18:11:32.507379 IP 10.0.0.1.67 > 10.0.0.4.68: BOOTP/DHCP, Reply, length 316


Now let's look at that first packet - a BOOTP/DHCP broadcast request on port 67

We'll specify options
- -e: to show the link level header also
- -v: to decode the DHCP request packet
- -c1: to display only the first matching packet
- dst port 67: to display packets destined to port 67

In [146]:
tcpdump -v -n -e -r dhcpProblem.capture.pcap dst port 67 -c1

reading from file dhcpProblem.capture.pcap, link-type EN10MB (Ethernet)
18:11:26.463826 fa:16:3e:01:85:d1 > ff:ff:ff:ff:ff:ff, ethertype IPv4 (0x0800), length 322: (tos 0x0, ttl 64, id 0, offset 0, flags [none], proto UDP (17), length 308)
    0.0.0.0.68 > 255.255.255.255.67: BOOTP/DHCP, Request from fa:16:3e:01:85:d1, length 280, xid 0xbb124533, Flags [none]
	  Client-Ethernet-Address fa:16:3e:01:85:d1
	  Vendor-rfc1048 Extensions
	    Magic Cookie 0x63825363
	    DHCP-Message Option 53, length 1: Discover
	    Client-ID Option 61, length 7: ether fa:16:3e:01:85:d1
	    MSZ Option 57, length 2: 576
	    Parameter-Request Option 55, length 7: 
	      Subnet-Mask, Default-Gateway, Domain-Name-Server, Hostname
	      Domain-Name, BR, NTP
	    Vendor-Class Option 60, length 12: "udhcp 1.18.5"


Note in the above output
- The source      MAC address is the broadcast address fa:16:3e:01:85:d1
- The destination MAC address is the broadcast address ff:ff:ff:ff:ff:ff
- The destination  IP address is the broadcast address 255.255.255.255
- This is a BOOTP/DHCP, Request

Now we can inspect the first DHCP reply send

In [148]:
tcpdump -v -n -e -r dhcpProblem.capture.pcap dst port 68 -c1

reading from file dhcpProblem.capture.pcap, link-type EN10MB (Ethernet)
18:11:26.464311 fa:16:3e:09:06:d9 > fa:16:3e:01:85:d1, ethertype IPv4 (0x0800), length 358: (tos 0x0, ttl 64, id 2832, offset 0, flags [none], proto UDP (17), length 344)
    10.0.0.1.67 > 10.0.0.4.68: BOOTP/DHCP, Reply, length 316, xid 0xbb124533, Flags [none]
	  Your-IP 10.0.0.4
	  Server-IP 10.0.0.1
	  Client-Ethernet-Address fa:16:3e:01:85:d1
	  Vendor-rfc1048 Extensions
	    Magic Cookie 0x63825363
	    DHCP-Message Option 53, length 1: Offer
	    Server-ID Option 54, length 4: 10.0.0.1
	    Lease-Time Option 51, length 4: 120
	    RN Option 58, length 4: 60
	    RB Option 59, length 4: 105
	    Subnet-Mask Option 1, length 4: 255.255.255.192
	    BR Option 28, length 4: 10.0.0.63
	    Default-Gateway Option 3, length 4: 10.0.0.1
	    Domain-Name-Server Option 6, length 4: 10.0.0.1
	    Domain-Name Option 15, length 9: "novalocal"
	    Hostname Option 12, length 11: "mycirros030"


Note in the above output
- The destination MAC address is the *original requestor* address fa:16:3e:01:85:d1
- The destination  IP address is the *offered* IP address 10.0.0.4
- This is a BOOTP/DHCP, Offer providing the requestor with
  - IP address 10.0.0.4
  - fully qualified host name (Host+Domain): mycirros030.novalocal



In [6]:
man tcpdump

bash: man: command not found


In [7]:
tcpdump --help

tcpdump version 4.6.2
libpcap version 1.6.2
OpenSSL 1.0.1k 8 Jan 2015
Usage: tcpdump [-aAbdDefhHIJKlLnNOpqRStuUvxX#] [ -B size ] [ -c count ]
		[ -C file_size ] [ -E algo:secret ] [ -F file ] [ -G seconds ]
		[ -i interface ] [ -j tstamptype ] [ -M secret ] [ --number ]
		[ -Q in|out|inout ]
		[ -r file ] [ -s snaplen ] [ --time-stamp-precision precision ]
		[ -T type ] [ --version ] [ -V file ]
		[ -w file ] [ -W filecount ] [ -y datalinktype ] [ -z command ]
		[ -Z user ] [ expression ]


<a name="tcpdump_manpage"/>
# tcpdump manpage
<a href="#TOP"> Go to TOP of page </a>



In [19]:
man tcpdump

bash: man: command not found


<a name="tcpdump_resources"/>
# Resources
<a href="#TOP"> Go to TOP of page </a>

- [**Packet Analyzer: 15 TCPDUMP Command Examples**
](http://www.thegeekstuff.com/2010/08/tcpdump-command-examples/)by SASIKALA on AUGUST 25, 2010


- [**Stackoverflow: How to filter tcpdump output based on a packet length**
](http://stackoverflow.com/questions/9874093/how-to-filter-tcpdump-output-based-on-a-packet-length)by Muhammet Can on Mar 26 2012


- [**Linux.com: Troubleshooting connection problems with TCPDUMP: DHCP**](https://www.linux.com/blog/troubleshooting-connection-problems-tcpdump-dhcp) by Shawn Holland, FEBRUARY 7, 2010

